# Monte Carlo notebook tour

This notebook walks through two ways to configure and visualise Monte Carlo analyses with Circuit Toolkit:
1. Build the circuit directly in Python.
2. Import an LTspice `.asc` schematic and run the same workflow.

Each section generates a small sample size and uses the interactive Plotly helpers. Tweak the number of trials or the SPICE engine as needed.

## Requirements

Install the visualisation extras and make sure a SPICE engine (for example `ngspice`) is on your PATH:

```bash
pip install circuit-toolkit[viz]
```

Update the `ENGINE` constant in the next cell if you prefer `ltspice` or `xyce`.

In [1]:
# Shared imports and helpers
from collections.abc import Iterable
from pathlib import Path

import numpy as np
from spicelab.analysis import NormalPct, monte_carlo
from spicelab.core.circuit import Circuit
from spicelab.core.components import Capacitor, Resistor, Vdc
from spicelab.core.net import GND, Net
from spicelab.core.types import AnalysisSpec
from spicelab.io.ltspice_asc import circuit_from_asc
from spicelab.viz import monte_carlo_histogram, monte_carlo_param_scatter

ENGINE = "ngspice"
TRANSIENT = AnalysisSpec("tran", {"tstep": 1e-5, "tstop": 2e-3})


def find_component_by_ref(circuit: Circuit, ref: str):
    for component in getattr(circuit, "_components", []):
        if getattr(component, "ref", None) == ref:
            return component
    raise KeyError(f"Component '{ref}' not found in the circuit")


def resolve_trace(result, target: str):
    target_lower = target.lower()
    for name in result.traces.names:
        if name.lower() == target_lower:
            return result.traces[name]
    available = ", ".join(result.traces.names)
    raise KeyError(f"Trace '{target}' not found. Available signals: {available}")


def metric_vout_final(result):
    trace = resolve_trace(result, "v(vout)")
    return {"vout_final": float(np.real(trace.values[-1]))}


def run_monte_carlo(
    circuit: Circuit,
    mapping: dict,
    *,
    n: int = 40,
    seed: int | None = None,
    analyses: Iterable[AnalysisSpec] | None = None,
):
    analyses = list(analyses) if analyses is not None else [TRANSIENT]
    return monte_carlo(
        circuit=circuit,
        mapping=mapping,
        n=n,
        seed=seed,
        analyses=analyses,
        engine=ENGINE,
    )


def show_mc_plots(samples, metrics, param_label: str, *, title_prefix: str):
    hist = monte_carlo_histogram(metrics, title=f"{title_prefix} – metric distribution")
    hist.show()
    scatter = monte_carlo_param_scatter(
        samples,
        metrics,
        param=param_label,
        title=f"{title_prefix} – impact of {param_label}",
        ylabel="vout_final",
    )
    scatter.show()

## Example 1 – circuit created in code

We build a simple RC filter via the core API and run Monte Carlo varying R1 and C1. The transient analysis captures the final value of `V(vout)` as the metric.

In [2]:
# Build the circuit in Python
code_circuit = Circuit("rc_code_mc")
vin = Net("vin")
vout = Net("vout")

v1 = Vdc("V1", 5.0)
r1 = Resistor("R1", "10k")
c1 = Capacitor("C1", "10n")

code_circuit.add(v1, r1, c1)
code_circuit.connect(v1.ports[0], vin)
code_circuit.connect(v1.ports[1], GND)
code_circuit.connect(r1.ports[0], vin)
code_circuit.connect(r1.ports[1], vout)
code_circuit.connect(c1.ports[0], vout)
code_circuit.connect(c1.ports[1], GND)

code_circuit

Circuit(name='rc_code_mc', _net_ids={}, _port_to_net={Port(owner=<Vdc V1 value=5.0>, name='p', role=<PortRole.POSITIVE: 1>): Net(name='vin'), Port(owner=<Vdc V1 value=5.0>, name='n', role=<PortRole.NEGATIVE: 2>): Net(name='0'), Port(owner=<Resistor R1 value='10k'>, name='a', role=<PortRole.POSITIVE: 1>): Net(name='vin'), Port(owner=<Resistor R1 value='10k'>, name='b', role=<PortRole.NEGATIVE: 2>): Net(name='vout'), Port(owner=<Capacitor C1 value='10n'>, name='a', role=<PortRole.POSITIVE: 1>): Net(name='vout'), Port(owner=<Capacitor C1 value='10n'>, name='b', role=<PortRole.NEGATIVE: 2>): Net(name='0')}, _components=[<Vdc V1 value=5.0>, <Resistor R1 value='10k'>, <Capacitor C1 value='10n'>], _directives=[], _subckt_defs={}, _subckt_instances=[])

In [3]:
# Generated netlist for reference
print(code_circuit.build_netlist())

* rc_code_mc
VV1 vin 0 5.0
RR1 vin vout 10k
CC1 vout 0 10n
.end



In [4]:
# Monte Carlo: vary R1 (5%) and C1 (10%)
code_mapping = {
    r1: NormalPct(0.05),
    c1: NormalPct(0.10),
}
code_mc = run_monte_carlo(code_circuit, code_mapping, n=80, seed=2024)
code_df = code_mc.to_dataframe(metric=metric_vout_final, param_prefix="param_")
code_df.head()

,param_Resistor.R1,param_Capacitor.C1,trial,vout_final
0,9207.078078,1.030158e-08,0,5.0
1,9653.825902,1.197147e-08,1,5.0
2,9329.347041,1.085017e-08,2,5.0
3,9964.190158,1.074658e-08,3,5.0
4,10225.014063,8.915110e-09,4,5.0


In [5]:
# Visualise the results
#!pip install nbformat==5.0.2
code_metrics = code_df["vout_final"].to_list()
show_mc_plots(code_mc.samples, code_metrics, "Resistor.R1", title_prefix="Code circuit")

ValueError: Too many bins for data range. Cannot create 50 finite-sized bins.

## Example 2 – circuit imported from an `.asc`

Now we load an LTspice schematic (`examples/rc_lowpass.asc`), convert it to a `Circuit`, run Monte Carlo, and reuse the same utilities. The `.asc` file was generated via Circuit Toolkit so it is compatible with the parser.

In [ ]:
# Load the LTspice schematic
asc_path = Path("rc_lowpass.asc")
lt_circuit = circuit_from_asc(asc_path)
lt_r1 = find_component_by_ref(lt_circuit, "R1")
lt_c1 = find_component_by_ref(lt_circuit, "C1")
print(f"Circuit imported from {asc_path}")
print(lt_circuit.build_netlist())

Circuit imported from rc_lowpass.asc
* ltspice_import
VV1 vin 0 5.0
RR1 vin vout 10k
CC1 vout 0 10n
.end



In [ ]:
# Monte Carlo on the imported circuit
lt_mapping = {
    lt_r1: NormalPct(0.05),  # 5% sigma around the nominal value
    lt_c1: NormalPct(0.10),
}
lt_mc = run_monte_carlo(lt_circuit, lt_mapping, n=80, seed=2025)
lt_df = lt_mc.to_dataframe(metric=metric_vout_final, param_prefix="param_")
lt_df.head()

,param_Resistor.R1,param_Capacitor.C1,trial,vout_final
0,10115.504341,9.653163e-09,0,5.0
1,9346.569427,9.547926e-09,1,5.0
2,9000.894805,1.064022e-08,2,5.0
3,9131.571313,9.891544e-09,3,5.0
4,9801.745640,1.082230e-08,4,5.0


In [ ]:
# Visualise the LTspice results
lt_metrics = lt_df["vout_final"].to_list()
show_mc_plots(lt_mc.samples, lt_metrics, "Resistor.R1", title_prefix="LTspice circuit")

### Next steps

- Tweak `TRANSIENT` to experiment with AC analyses or sample traces at specific times (`y=[...]` with `sample_at`).
- Swap `metric_vout_final` for a function that computes gain, rise time, or any KPI relevant to your circuit.
- For `.cir` netlists use `spicelab.io.ltspice_parser.from_ltspice_file` and reuse the same helpers above.